In [2]:
import svgutils.compose as sc
from IPython.display import SVG
from types import SimpleNamespace
import numpy as np
import holoviews as hv
hv.extension('matplotlib', logo=False) #BOKEH DOESNT SUPPORT LATEX IN LABELS
from holoviews import opts
hv.output(fig='svg')

import kwant
import numpy as np
# import tinyarray as tiny
from types import SimpleNamespace

C:\ProgramData\Anaconda3\envs\lps\lib\site-packages\kwant\solvers\default.py:18: RuntimeWarning: MUMPS is not available, SciPy built-in solver will be used as a fallback. Performance can be very poor in this case.
  "Performance can be very poor in this case.", RuntimeWarning)


# Goal
#### Calculate transport properties of skyrmion with Kwant

## To do
- Write explicit 16x16 basis of $\Psi(r)\Psi(r+1)$ to include hopping terms and to check if the defined matrices give the right terms for the Hamiltonian of the skyrmion paper
- Program skyrmion in Kwant
- Read *Aguado* Review
- Export holoviews as GIF

## 14 May
- Finished the magnetic texture of the skyrmion, although plotting it with plt.quiver which plots the vector field with arrows was very annoying, so to check, I just made it such that you can check the projection of the magnetic field in the three directions:

In [3]:
def magn_texture(position,params):
    x,y = position
    t = np.arctan2(y,x)
    p = params.aziwinding
    q = params.radiwinding
    R = params.radius
    r = np.sqrt(x**2 + y**2)
    B = [np.sin(2*np.pi*p*(r/R))*np.cos(q*t), np.sin(2*np.pi*p*(r/R))*np.sin(q*t), np.cos(2*np.pi*q*(r/R))]
    return B
params = SimpleNamespace(t=1, mu=-0.1, j=0.02, delta=0.2, B_0=0.1, aziwinding=1, radiwinding=2,radius=15)
init = []
x = np.arange(-params.radius,params.radius,1)
y = np.arange(-params.radius,params.radius,1)
for i in x:
    for j in y:
        init.append(magn_texture((i,j),params))
B = np.reshape(np.asarray(init),(len(x),len(y),3));
hv.HoloMap({i: hv.Image(B[:,:,i], bounds=(x[0],y[0],-x[0],-y[0])).opts(colorbar=True, cmap='inferno') for i in range(3)}, kdims=('Projection Axis'))

:HoloMap   [Projection Axis]
   :Image   [x,y]   (z)

## 13 May
- kwant.Builder doesn't seem to like it yet when you want to pass a dictionary to the params parameter, so you have to use args[p], where p can be a dictionary/simplenamespace. There is an error when trying to define the hopping terms on the disk -> dimension of onsite terms doesn't match those of the hopping ones.
- A bit of confusion regarding the Hamiltonian as defined in the skyrmion paper: 
\begin{equation}
H(\mathbf{r}) =   \left(  - \frac{1}{2m}\nabla^2 - \mu \right)\tau_z + J \sigma_{\mathbf{r}} \cdot \mathbf{B(r)} + \Delta_0 \tau_x.
\end{equation}
Using the Nambu basis
\begin{equation}
\psi^{\dagger}(\mathbf{r}) = \left( c^{\dagger}_{\uparrow}(\mathbf{r}) c^{\dagger}_{\downarrow}(\mathbf{r}) \ 
    c_{\downarrow}(\mathbf{r}) -c_{\uparrow}(\mathbf{r}) \right), 
\end{equation}
how are the (4x4 matrices) $\tau_z$, $\tau_x$ and $\sigma_x,\sigma_y,\sigma_z$ defined? 
We know from the Hamiltonian that $\tau_z$ counts number of particles so $\tau_z = \sigma_z \otimes I_2$ and $\sigma_z$ counts the number of up- and down-spin particles (with + for up and - for down), so $\sigma_z = I_2 \otimes \sigma_z$. However, saying that $\sigma_y = I_2 \otimes \sigma_y$ and $\sigma_x = I_2 \otimes \sigma_x$, would be wrong because of the minus sign in the definition of the Nambu basis. Including the minus sign (and knowing that $\sigma_x$ should flip the spin of the operators), can I infer that $\sigma_x = \sigma_z \otimes \sigma_x$ (and $\sigma_y = I_2 \otimes \sigma_y$)?
Likewise, including the minus sign and knowing that $\tau_x$ should swap the electron and hole operators (leaving the spin sign untouched), can we say that $\tau_x = \sigma_y \otimes \sigma_y$? 
Then $\psi^{\dagger} \Delta_0 \tau_x \psi = \Delta_0 (c^{\dagger}_{\uparrow}c^{\dagger}_{\downarrow} + c^{\dagger}_{\downarrow}c^{\dagger}_{\uparrow} + \mathrm{h.c.} )$ which would be zero because of the anticommutation relation $\{c^{\dagger}_{\uparrow},c^{\dagger}_{\downarrow} \}=0$, so $\tau_x$ has to be different (something like $\sigma_x \otimes I_2$?).

## 10 May
 - Finished the Kwant whitepaper
 - Playing around with Kwant on a <a href='https://nbviewer.jupyter.org/github/Arfor/m1internship/blob/master/Jupyter/Kwant_disk.ipynb'> disk</a> -> Made basic setup for magnetic texture on disk and defined function to fill onsite terms of Hamiltonian
 ![Disks](Jupyter/fig/disks_r=10.svg)


## 9 May
 - Made sure the jupyter notebooks are available to view online by making a repo on GitHub
 - Finished 1st read of Skyrmion paper
 - Started reading the <a href='https://arxiv.org/abs/1309.2926'> whitepaper of kwant </a> (2014)
 - Got circular geometry on a square lattice structure using kwant.Builder


## 7 May
- Started reading *Topological Superconductivity with Deformable Magnetic Skyrmions - Garnier, Mesaros, Simon*

The reason why such a complicated magnetic texture is being researched is because it is a way to achieve p-wave coupling in a superconductor, which is reducible to Kitaev's model. Only in p-wave superconductors we find topological invariants and Majorana edge modes, but we have no evidence as of now that they exist in nature. Therefore, p-waves must be engineered and using the magnetic texture of a skyrmion, which has already been made in the lab, is one possible way of achieving this. 
The crux to achieving p-wave superconductivity is to deform the Hamiltonian in such a way that the only pairing that can happen at a certain energy is between two spin-independent electrons:
$$
\Delta_s c^{\dagger}_{k\uparrow} c^{\dagger}_{-k,\downarrow} \to \Delta_p c^{\dagger}_{k}c^{\dagger}_{-k}
$$
However, just splitting the spin values with for example a Zeeman term in the z-direction is not enough. For example, a simple Zeeman term in the z-direction would split the electrons into up- and down-spin, but since we proximitize the skyrmion with an s-wave superconductor, the electrons cannot be coupled all of sudden with the same spin (p-wave). <br>
*Not too sure about this: If two Kitaev chains with Majorana edges would be close to each other, they would be able to talk and destroy the zero-energy modes. For a simple Zeeman term there would be two copies of the Kitaev chain for a certain energy $\epsilon$ in close proximity which would destroy the zero-mode.*
$$H_B = -B_z \sigma_z c^{\dagger}_{k\sigma}c^{}_{k\sigma}$$

In [4]:
k = np.linspace(-np.pi,np.pi,200)
e_up = k**2 - 1
e_down = k**2 + 1
left,right = (k-1)**2, (k+1)**2

plot1 = hv.Overlay([hv.Path((k,e_up), kdims=['$k_z$','$\epsilon$']),hv.Path((k,e_down)),hv.HLine(0).opts(linestyle='--',color='grey'),hv.VLine(0).opts(linestyle='--', color='grey')])


plot2 = hv.Overlay([hv.Curve((k,left), kdims=['$k_x$','$\epsilon$']),hv.Curve((k,right)),hv.HLine(0).opts(linestyle='--',color='grey'),hv.VLine(0).opts(linestyle='--', color='grey')])
plot1.opts(yticks=[(0,0)],xticks=[(-np.pi, '$-\pi$'),(0, 0), (np.pi, '$\pi$')],ylim=(-2,6)) + plot2.opts(yticks=[(0,0)],xticks=[(-np.pi, '$-\pi$'),(0, 0), (np.pi, '$\pi$')], ylim=(0,6))

:Layout
   .Overlay.I  :Overlay
      .Path.I  :Path   [$k_z$,$\epsilon$]
      .Path.II :Path   [x,y]
      .HLine.I :HLine   [x,y]
      .VLine.I :VLine   [x,y]
   .Overlay.II :Overlay
      .Curve.I  :Curve   [$k_x$,$\epsilon$]   (y)
      .Curve.II :Curve   [x]   (y)
      .HLine.I  :HLine   [x,y]
      .VLine.I  :VLine   [x,y]

**_Question_**: *If we go to $\epsilon=0$ now, wouldn't you get p-wave coupling?* <br>
**_Answer_**: *No, proximitizing with an s-wave superconductor makes it impossible to immediately get coupling of electrons with equal spins*

So we have to add some spices in order to be able to get nearer to the Kitaev model --> magnetic field dependence in 3 directions
$$H_B = -B \vec{\sigma} \cdot \vec{n}$$

Now, the Zeeman splitting should create an energy gap at $k_x=0$, such that there are again two (helicity) bands which are split from each other. In the band-gap, the electrons with opposite $k$ can couple like p-waves. However, I'm not too sure about why this happens, I'll need to understand the *Aguado* review to understand the Rashba effect which enable the p-wave coupling.

## 6 May
- Finished Kitaev Chain using Kwant
- Made everything interactive with Holoviews

See <a href='https://nbviewer.jupyter.org/github/arfor/m1internship/blob/master/Jupyter/Kitaev%20Chain.ipynb'>Kitaev Chain.ipynb</a>

## 3 May
- Model Plot of Kitaev Chain just with Python
- Same model using Kwant

Whenever there is a conservation law (symmetry/invariance: time->energy, translation->momentum, rotation->anular momentum), the hamiltonian can be block diagonalised.
Then one can study the topology of the blocks independently. In sublattice symmetry, the number of levels above and below the fermi level are constant and there are no zero-energy crossings -> topologically invariant.
In particle-hole symmetry (e.g. superconductors), also the number of eigenvalues below and above zero are equal, but there exist zero-energy crossings -> there is some topology, but obviously, the number of eigenvalues below zero is not a good topological number since it doesn't change. The right topological invariant is the signature of the *Pfaffian* of the Hamiltonian, which is -1 for odd number of electrons and +1 for an even number.

> **The symmetry of the system determines the type and existence of a topological invariant**
